In [ ]:
import os
import pandas as pd # for data cleaning and manipulation
import numpy as np # for data cleaning and manipulation
import googlemaps # for distance matrix API
import geopandas as gpd # for spatial analysis
from shapely import wkt, wkb

import subprocess # for ORSM
import json # for ORSM
import random # for ORSM

from pyspark.sql import SparkSession # for Apache Spark
from sedona.spark import * # for Apache Sedona
from sedona.register import SedonaRegistrator # for Apache Sedona
from sedona.utils import SedonaKryoRegistrator, KryoSerializer # for Apache Sedona

# Data Cleaning - Datasets A, B1 and B2

In [2]:
def list_excel_files(directory='.'):
    files = []
    for i,j,k in os.walk(directory):
        for file in k:
            if file.endswith('.xlsx'):
                files.append(file)
    return files

In [3]:
files = list_excel_files()

In [4]:
def create_dataframe(file_list, prefix):
    df = pd.concat(pd.read_excel(file, parse_dates=False) for file in file_list if prefix in file)
    return df

In [5]:
matos = create_dataframe(files, 'matos')

In [6]:
matos_ec = matos[(matos['species'] == 'Ec') & (matos['year'] == 2026)]

In [7]:
sim = create_dataframe(files, 'sim')

In [8]:
sim_ec = sim[(sim['species'] == 'Ec') & (sim['Year'] == 2026)]

In [9]:
matos_ec.rename(columns={'id_stand': 'UG'}, inplace=True)

/tmp/ipykernel_1960/722918832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matos_ec.rename(columns={'id_stand': 'UG'}, inplace=True)


In [10]:
merged_df = pd.merge(matos_ec, sim_ec, on='UG', how='outer')

In [13]:
columns = ['UG','id_presc','t_x','v_thin','v_harv','si','fmm_x','avgdbh','g','resp','pburn','psd','pd','r','costshrremha','costswithshrubsremoval','fmm_y','areafmm_y','Area','Presc','Year','Ttotal','t_y','Rot','Nst','N','Ndead','V','Vserr','Vrol','Vres','Vthin','VHarv']

In [14]:
df = merged_df[columns]

In [16]:
b2 = df

In [17]:
b2_worst = b2.loc[b2.groupby('UG')['pburn'].idxmax()]

In [19]:
b2_best = b2.loc[b2.groupby('UG')['pburn'].idxmin()]

In [14]:
df = df.astype(str)

In [15]:
df.to_parquet('data_0922.parquet')

# PySpark

https://www.bing.com/search?q=setting+up+apache+sedona+locally&cvid=7ed5cf284536482f932e02c1c1e9574e&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEDSAQg1MzIzajBqOagCCLACAQ&FORM=ANAB01&PC=U531

In [10]:
# os.environ['JAVA_OPTS'] = "-Xmx8G"
# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [15]:
config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-3.3_2.12:1.7.2,"
        "org.datasyslab:geotools-wrapper:1.7.2-28.5",
    )
    .config(
        "spark.jars.repositories",
        "https://artifacts.unidata.ucar.edu/repository/unidata-all",
    )
    .getOrCreate()
)
sedona = SedonaContext.create(config)

Picked up _JAVA_OPTIONS: -Xms4g -Xmx8g
Picked up _JAVA_OPTIONS: -Xms4g -Xmx8g
25/09/29 14:14:49 WARN Utils: Your hostname, FW-Server resolves to a loopback address: 127.0.1.1; using 172.18.62.20 instead (on interface eth0)
25/09/29 14:14:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/gergely_soptei/.ivy2/cache
The jars for the packages stored in: /home/gergely_soptei/.ivy2/jars
org.apache.sedona#sedona-spark-3.3_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b59923ad-5b6e-4e4a-a942-3f8e252a4ebf;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/gergely_soptei/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.3_2.12;1.7.2 in central
	found org.apache.sedona#sedona-common;1.7.2 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

In [16]:
spark = SparkSession.builder \
    .appName("Large DF") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config("spark.jars.packages", ",".join([
        "org.apache.sedona:sedona-python-adapter-3.4_2.12:1.5.1",
        "org.datasyslab:geotools-wrapper:1.1.0-25.2"
    ])) \
    .getOrCreate()

25/09/29 14:15:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [26]:
from_parquet = spark.read.parquet("data_0922.parquet")

In [27]:
from_parquet.show()

25/09/29 09:41:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+--------+---+------+------+---+-----+-------------------+-------------------+----+-----------+-----------+-----------+-----------+------------+----------------------+-----+---------+-----+-----+----+------+---+---+-----+-----+-----+-------+-----+----+----+-----+-----+
| UG|id_presc|t_x|v_thin|v_harv| si|fmm_x|             avgdbh|                  g|resp|      pburn|        psd|         pd|          r|costshrremha|costswithshrubsremoval|fmm_y|areafmm_y| Area|Presc|Year|Ttotal|t_y|Rot|  Nst|    N|Ndead|      V|Vserr|Vrol|Vres|Vthin|VHarv|
+---+--------+---+------+------+---+-----+-------------------+-------------------+----+-----------+-----------+-----------+-----------+------------+----------------------+-----+---------+-----+-----+----+------+---+---+-----+-----+-----+-------+-----+----+----+-----+-----+
|  6|      92| 14|   0.0|     0|  4|  4.0|               16.4|2025-07-06 00:00:00|   0|0.009169859|0.244768815|0.815668491|0.998169236|         250|                   0.0|  4.0| 

In [28]:
a = from_parquet

In [29]:
a = a.filter(((a.t_y == 10) | (a.t_y == 11) | (a.t_y == 12)) & (a.VHarv != 0.0) & (a.VHarv == a.v_harv))

In [30]:
a.count()

193737

In [31]:
a.show()

+---+--------+---+------+------+---+-----+------+-------------------+----+------------------+-----------------+------------------+------------------+------------+----------------------+-----+---------+----+-----+----+------+---+---+----+----+-----+-------+------+------+----+-----+------+
| UG|id_presc|t_x|v_thin|v_harv| si|fmm_x|avgdbh|                  g|resp|             pburn|              psd|                pd|                 r|costshrremha|costswithshrubsremoval|fmm_y|areafmm_y|Area|Presc|Year|Ttotal|t_y|Rot| Nst|   N|Ndead|      V| Vserr|  Vrol|Vres|Vthin| VHarv|
+---+--------+---+------+------+---+-----+------+-------------------+----+------------------+-----------------+------------------+------------------+------------+----------------------+-----+---------+----+-----+----+------+---+---+----+----+-----+-------+------+------+----+-----+------+
| 21|      21| 11|   0.0|347.11|  3|  4.0|  13.5|2025-06-06 00:00:00|   0|       0.070208756|      0.246659561|       0.753882086|   

In [32]:
a = a.toPandas()

In [33]:
a.UG = a.UG.astype(int)

In [35]:
vds = gpd.read_file('./shape/shape/ValedoSousa_2024_withcentroids.shp')

In [36]:
vds.rename(columns={'ID_GENERAL': 'UG'}, inplace=True)

In [38]:
b2_worst = pd.merge(b2_worst, vds, on='UG', how='inner')

In [39]:
b2_worst = gpd.GeoDataFrame(b2_worst)

In [51]:
b2_worst.to_file('/mnt/c/Users/gergely.soptei.FORESTWISE/Documents/b2_worst.shp')

/tmp/ipykernel_177700/2270938214.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  b2_worst.to_file('/mnt/c/Users/gergely.soptei.FORESTWISE/Documents/b2_worst.shp')
/home/gergely_soptei/sedona_env/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'costshrremha' to 'costshrrem'
  ogr_write(
/home/gergely_soptei/sedona_env/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'costswithshrubsremoval' to 'costswiths'
  ogr_write(


In [40]:
b2_best = pd.merge(b2_best,vds, on='UG', how='inner')

In [41]:
b2_best = gpd.GeoDataFrame(b2_best)

In [90]:
b2_best.to_file('/mnt/c/Users/gergely.soptei.FORESTWISE/Documents/b2_best.shp')

/tmp/ipykernel_2304/2363018568.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  b2_best.to_file('/mnt/c/Users/gergely.soptei.FORESTWISE/Documents/b2_best.shp')
/home/gergely_soptei/sedona_env/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'costshrremha' to 'costshrrem'
  ogr_write(
/home/gergely_soptei/sedona_env/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'costswithshrubsremoval' to 'costswiths'
  ogr_write(


In [42]:
merged_geo_df = pd.merge(a, vds, on='UG', how='inner')

In [43]:
geo_df = gpd.GeoDataFrame(merged_geo_df)

In [45]:
a.drop_duplicates(subset='UG', inplace=True)

In [47]:
vale = gpd.read_file('ValedoSousa_2024_withcentroids.shp')

In [48]:
vale.rename(columns={"ID_GENERAL": "UG"}, inplace=True)

In [49]:
vale.crs

<Projected CRS: EPSG:3763>
Name: ETRS89 / Portugal TM06
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Portugal - mainland - onshore.
- bounds: (-9.56, 36.95, -6.19, 42.16)
Coordinate Operation:
- name: Portugual TM06
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [63]:
locations = vale.merge(a, on='UG', how="inner")

In [65]:
locations = gpd.GeoDataFrame(locations)

In [21]:
# locations.to_file("a_0910.shp")

In [67]:
locations["centroid"] = locations['geometry'].centroid

In [69]:
locations = locations['centroid'].to_crs(4326)

In [70]:
locations = gpd.GeoDataFrame(locations)

In [72]:
locations = [(point.x, point.y) for point in locations['centroid']]

# Open-Source Routing Machine

In [74]:
subset = random.sample(locations, 100)

In [76]:
subset = [ str(j) for i in subset for j in i]

In [78]:
url = f"http://router.project-osrm.org/table/v1/driving/{';'.join([','.join(x) for x in zip(subset[0::2],subset[1::2])])}?annotations=distance,duration"

In [79]:
url

'http://router.project-osrm.org/table/v1/driving/-8.240739303431356,41.00632496704199;-8.32586535615524,41.11065147657261;-8.276604911895364,40.95820316026571;-8.25907976968054,40.96903509713349;-8.346753026728583,41.143409101465814;-8.28428797114764,41.00590251880919;-8.286175327349726,41.06032544233441;-8.237343014029149,41.00864471049693;-8.312592476795018,40.95529817536023;-8.28403940143011,40.99875790106325;-8.334116800664948,41.14172481248837;-8.2410651219917,41.02210272912646;-8.247029825487667,40.98255381553383;-8.37018713963333,41.113822188882985;-8.340211095463786,41.12115968671534;-8.362642880399068,41.128462474736054;-8.223722398420874,41.007083941232146;-8.392964351425183,41.100699425561956;-8.28551101667646,40.9797746221533;-8.26590062299589,40.9582993196898;-8.334813262292815,41.11844484213817;-8.315369630609938,41.08048525268862;-8.355170647291441,41.07975049778602;-8.273692029537804,41.05525354667106;-8.279301604106442,41.00348592982176;-8.419866839966856,41.1033677804

In [80]:
combined_responses = [] 
try:
    curl_command = ['curl', url]
    result = subprocess.check_output(curl_command)
    # response_json = json.loads(result.decode('utf-8'))
    response_json = json.loads(result)
    combined_responses.append(response_json)
    print(f'Lenght of responses: {len(combined_responses)}')
except Exception as e:
    print(f'Something went wrong: {e}')        
else:
    print("We have reached the end")
with open("combined_0929.json", "w") as file:
    json.dump(combined_responses, file, indent=4)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Lenght of responses: 1
We have reached the end


100  176k  100  176k    0     0   222k      0 --:--:-- --:--:-- --:--:--  222k


# Optimal Solution

In [82]:
df = pd.read_excel('Solution_IA_2_7_LP_Test_MAXTIMB_26_May_25.xlsx', sheet_name='Sheet2')

In [83]:
df

,name2
0,Presc318_Pa26_Ct_10
1,Presc308_Pa50_Ct_0
2,Presc318_Pa176_Ct_10
3,Presc309_Pa337_Ct_0
4,Presc310_Pa344_Ct_10
...,...
2567,Presc801_Pa3206_Rp_0
2568,Presc801_Pa3207_Rp_0
2569,Presc801_Pa3208_Rp_0
2570,Presc801_Pa3209_Rp_0


In [84]:
prescriptions = [p for p in list(df.name2.dropna()) if 'Presc' in p and 'Ec' in p]

In [86]:
split_prescriptions = [p.split('_') for p in prescriptions]

In [88]:
ugs_with_shrubs = [[int(i[0][5:]),int(i[1][2:]),int(i[3])] for i in split_prescriptions]

In [90]:
headers = ["Presc", "UG", "Shrub"]

In [91]:
optimal_df = pd.DataFrame(ugs_with_shrubs, columns=headers)

In [94]:
import geopandas as gpd

In [95]:
gdf = gpd.read_file('ValedoSousa_2024_withcentroids.shp')

In [96]:
gdf.rename(columns={"ID_GENERAL": "UG"}, inplace=True)

In [99]:
optimal_gdf = a.merge(optimal_df, on='UG', how="inner")

In [100]:
optimal_gdf = optimal_gdf.merge(gdf, on='UG', how='inner')

In [101]:
optimal_gdf = gpd.GeoDataFrame(optimal_gdf)

In [102]:
optimal_gdf.UG = optimal_gdf.UG.astype(int)

In [22]:
# optimal_gdf.to_file('/mnt/c/Users/gergely.soptei.FORESTWISE/Documents/a_0923.shp')

# Google Maps API

In [ ]:
gmaps = googlemaps.Client()

In [3]:
cities = ["Lisboa", "Porto", "Coimbra"]

In [4]:
def get_distance_matrix(locations):
    distances = []
    for origin in locations:
        row = []
        for destination in locations:
            matrix = gmaps.distance_matrix(origin, destination, mode="driving")
            distance = matrix["rows"][0]["elements"][0]["distance"]["value"]
            row.append(distance)
        distances.append(row)
    return pd.DataFrame(distances,index=locations, columns=locations)

In [7]:
distance_matrix = get_distance_matrix(cities)

In [8]:
distance_matrix

,Lisboa,Porto,Coimbra
Lisboa,0,312147,202685
Porto,313103,0,123723
Coimbra,205955,121574,0


# Sedona

In [ ]:
vale = sedona.createDataFrame(vale)

/home/gergely_soptei/sedona_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:428: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Did not pass numpy.dtype object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [ ]:
vale.show()

+----------+-------------+-------------+-------------+--------------+-------------+--------------------+
|ID_GENERAL|   Shape_Leng|   Shape_Area|      Area_ha|            Xx|           YY|            geometry|
+----------+-------------+-------------+-------------+--------------+-------------+--------------------+
|         1|756.496135708| 29852.230502| 2.9852230502|-17250.4009257|165334.537418|POLYGON ((-17257....|
|         2|3807.87921802|284793.301126|28.4793301126|-14164.1658327|163023.796556|POLYGON ((-13834....|
|         3|3473.76718094|65089.7120123|6.50897120123|-7647.27088631|152530.779793|POLYGON ((-7537.1...|
|         4|868.157366989|23482.3507736|2.34823507736|-7366.82161511|153113.845834|POLYGON ((-7261.8...|
|         5|965.498555625|27698.7367216|2.76987367216|-7386.68713871| 152330.73611|POLYGON ((-7261.8...|
|         6|1705.29120949|113222.389708|11.3222389708|-8046.46680077|152606.814289|POLYGON ((-8001.6...|
|         7| 2141.1894357|136237.851887|13.6237851887|-

In [ ]:
unique_id = [i['ID_GENERAL'] for i in vale.select('ID_GENERAL').distinct().collect()]

25/07/31 11:00:25 WARN TaskSetManager: Stage 27 contains a task of very large size (1037 KiB). The maximum recommended task size is 1000 KiB.
